In [1]:
# Import library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import datetime
import string
import random
import warnings

from scipy import stats
from matplotlib import colors


#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim 
import seaborn as sns

import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords

warnings.filterwarnings("ignore")

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\ProgramData\Anaconda3\lib\site-packages\nose\config.py:178: DeprecationWarning: invalid escape sequence \.
  """
C:\ProgramData\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\ProgramData\Anaconda3\lib\site-packages\nltk\featstruct.py:1296: DeprecationWarning: invalid escape sequence \d
  name, n = re.sub('\d+$', '', var.name), 2
C:\ProgramData\Anaconda3\lib\site-packages\nltk\featstruct.py:2092: DeprecationWarning: invalid escape sequence \d
  RANGE_RE = re.compile('(

C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:518: DeprecationWarning: invalid escape sequence \g
  RegexpChunkRule.__init__(self, regexp, '\g<chunk>', descr)
C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:583: DeprecationWarning: invalid escape sequence \g
  RegexpChunkRule.__init__(self, regexp, '\g<left>', descr)
C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:718: DeprecationWarning: invalid escape sequence \{
  '(?P<left>%s)\{(?P<right>%s)'
C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:724: DeprecationWarning: invalid escape sequence \g
  RegexpChunkRule.__init__(self, regexp, '{\g<left>\g<right>', descr)
C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:789: DeprecationWarning: invalid escape sequence \}
  '(?P<left>%s)\}(?P<right>%s)'
C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:795: DeprecationWarning: invalid escape sequence \g
  RegexpChunkRule.__init__(self, regexp, '\g<left

In [2]:
filename = '.\ResearchJournals_Senescence.csv'

In [3]:
df_text = pd.read_csv(filename, sep='\t')

In [4]:
# dictionary = corpora.Dictionary(simple_preprocess(line, deacc=True) for line in open(filename, encoding='utf-8'))
# corpus = [dictionary.doc2bow(simple_preprocess(line)) for line in open(filename, encoding='utf-8')]

In [5]:
# tfidf = models.TfidfModel(corpus, smartirs='ntc')

In [8]:
def strip_punct(in_str):
    # Remove punctuation from string.
    punct_filt = string.punctuation
    trans = str.maketrans('','', punct_filt)
    return in_str.translate(trans)

def make_processed_sentence(in_str):
    # Remove punctuation and split string into list of words  
    no_punct_lc = strip_punct(str(in_str))
    word_list = no_punct_lc.split(' ')
    return word_list

def remove_stopwords(str_list):
    # Remove trivial words from list of words.
    stopwords_list = stopwords.words('english')
    no_stopwords = []
    for sentence in str_list:
        no_stopwords.append([word for word in sentence if word not in stopwords_list])
    return no_stopwords    
    
    
def build_bigram_model(proc_text_list, m_c, thresh):
    bigram = gensim.models.Phrases(proc_text_list, min_count=m_c, threshold=thresh) 
    return bigram 

def apply_bigram_model(bigram):
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

def make_bigrams(review_list, bigram_mod):
    bigrams = [bigram_mod[review] for review in review_list]
    return bigrams

def lemmatization(review_list, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'], disallowed_postags=['PROPN']):
    """https://spacy.io/api/annotation"""
    reviews_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for review in review_list:
        rev = nlp(" ".join(review)) 
        reviews_out.append([token.lemma_ for token in rev 
                            if token.pos_ in allowed_postags 
                            and token.pos_ not in disallowed_postags])
    return reviews_out

def build_lda_model(text_df, text_column):
    proc_text_list = []
    for review in text_df[text_column]:
        proc_text_list.append(make_processed_sentence(review))

    # Remove Stop Words
    review_words_nostops = remove_stopwords(proc_text_list)

    # Form Bigrams
    bigram=build_bigram_model(proc_text_list, 5, 25)
    bigram_mod=apply_bigram_model(bigram)
    review_words_bigrams = make_bigrams(review_words_nostops, bigram_mod)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en

    # Do lemmatization keeping only noun, adj, vb, adv
    reviews_lemmatized = lemmatization(review_words_bigrams, 
                                       allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'], 
                                       disallowed_postags=['PROPN'])

    # Create Dictionary
    id2word = corpora.Dictionary(reviews_lemmatized)
    id2word.filter_extremes(no_below=30, no_above=0.30)

    # Create Corpus
    review_texts = reviews_lemmatized

    # Term Document Frequency
    texts_bow = [id2word.doc2bow(text) for text in review_texts]


    lda_model = gensim.models.ldamodel.LdaModel(corpus=texts_bow,
                                               id2word=id2word,
                                               num_topics=15, 
                                               random_state=102,
                                               update_every=1,
                                               chunksize=1000,
                                               passes=15,
                                               alpha='auto',
                                               per_word_topics=True)

    # Compute Perplexity. Lower scores are better.
    print('\nPerplexity: ', lda_model.log_perplexity(texts_bow))  

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews_lemmatized, 
                                         dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)

    # Visualize the topics
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model, texts_bow, id2word)
    return lda_model, vis, coherence_lda

In [9]:
lda_model, vis, coherence = build_lda_model(df_text, 'ArticleText')


Perplexity:  -8.383338743687496

Coherence Score:  0.6441659851713253


In [10]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.044788 -0.127381       1        1  10.928245
2      0.098979 -0.167278       2        1  10.107724
6      0.089084  0.180077       3        1   8.653070
5      0.000690  0.121992       4        1   7.801888
11     0.219332 -0.016154       5        1   7.757739
9      0.255241 -0.099765       6        1   7.731594
14    -0.133700 -0.152843       7        1   7.521649
7      0.089408 -0.004171       8        1   7.391021
4     -0.080892 -0.000372       9        1   6.544816
10    -0.047976 -0.056941      10        1   6.127742
1     -0.068060  0.216561      11        1   5.143394
8     -0.166775 -0.070393      12        1   4.585584
0      0.042030  0.073495      13        1   4.082948
13    -0.079103  0.198338      14        1   3.841158
3     -0.173470 -0.095166      15        1   1.781420, topic_info=      Category          Freq                  Term         Total  loglift  \
872    Default  63634.000000                 tumor  63634.000000  30.0000   
42     Default  37350.000000                 brain  37350.000000  29.0000   
428    Default  24893.000000                  bone  24893.000000  28.0000   
1353   Default  37895.000000                     β  37895.000000  27.0000   
214    Default  24287.000000         mitochondrial  24287.000000  26.0000   
177    Default  18888.000000               insulin  18888.000000  25.0000   
398    Default  24896.000000           antioxidant  24896.000000  24.0000   
525    Default  18535.000000           endothelial  18535.000000  23.0000   
1178   Default  19676.000000                muscle  19676.000000  22.0000   
2056   Default  17783.000000                neuron  17783.000000  21.0000   
980    Default  24611.000000              compound  24611.000000  20.0000   
681    Default  20895.000000              mutation  20895.000000  19.0000   
1083   Default  19362.000000               glucose  19362.000000  18.0000   
943    Default  14383.000000               cardiac  14383.000000  17.0000   
1411   Default  16064.000000              diabetic  16064.000000  16.0000   
5257   Default  20818.000000                  lung  20818.000000  15.0000   
331    Default  16889.000000                  stem  16889.000000  14.0000   
608    Default  18841.000000                kidney  18841.000000  13.0000   
296    Default  17574.000000                 renal  17574.000000  12.0000   
1277   Default  19533.000000              sequence  19533.000000  11.0000   
3008   Default  19322.000000            macrophage  19322.000000  10.0000   
364    Default  14626.000000              vascular  14626.000000   9.0000   
1065   Default  10691.000000              exercise  10691.000000   8.0000   
731    Default  16029.000000                 plant  16029.000000   7.0000   
1022   Default  12081.000000                  diet  12081.000000   6.0000   
213    Default  13872.000000          mitochondria  13872.000000   5.0000   
1700   Default  24587.000000                lesion  24587.000000   4.0000   
123    Default   7864.000000                   eye   7864.000000   3.0000   
261    Default  16548.000000       phosphorylation  16548.000000   2.0000   
4253   Default  20141.000000                medium  20141.000000   1.0000   
...        ...           ...                   ...           ...      ...   
9012   Topic15   2199.015381                cornea   2215.087158   4.0205   
9013   Topic15   2623.218506               corneal   2658.378174   4.0144   
6937   Topic15   3987.894043                 sperm   4068.189209   4.0078   
123    Topic15   7481.822754                   eye   7864.851074   3.9778   
4849   Topic15    831.713989         visual_acuity    840.260315   4.0175   
8288   Topic15   2146.362061                ocular   2222.555908   3.9929   
22581  Topic15    822.206970                 seman    835.418579   4.0118   
1603   Topic15   1619.994019        